In [5]:
import os
import pandas as pd
import numpy as np
from numpy import expand_dims
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from skimage.transform import resize
from IPython.display import SVG
from tensorflow import keras
from keras import applications
from keras import optimizers
from keras.models import Model, Sequential
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.layers import Dense, Flatten, Dropout, BatchNormalization, GlobalAveragePooling2D
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.utils.np_utils import to_categorical
from keras import layers, models
from keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau
from google.colab import drive

drive.mount('/content/drive')

train_path = "/content/drive/My Drive/Colab Notebooks/ml/images/train/"
test_path = "/content/drive/My Drive/Colab Notebooks/ml/images/test/"
img_width, img_height = 224, 224
channels = 3
batch_size = 64
num_images = 100
image_arr_size = img_width * img_height * channels


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:

train_datagen = ImageDataGenerator(
    rescale= 1./255,
    shear_range= 0.2,
    zoom_range= 0.2,
    horizontal_flip= True,
    rotation_range= 20,
    width_shift_range= 0.2,
    height_shift_range= 0.2,
    validation_split=0.2,

)

valid_datagen = ImageDataGenerator(
    rescale= 1./255,
    validation_split=0.2,
)


train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size= (img_width, img_height),
    color_mode= 'rgb',
    batch_size= batch_size,
    class_mode= 'categorical',
    subset='training',
    shuffle= True,
    seed= 1337
)

valid_generator = valid_datagen.flow_from_directory(
    train_path,
    target_size= (img_width, img_height),
    color_mode= 'rgb',
    batch_size= batch_size,
    class_mode= 'categorical',
    subset='validation',
    shuffle= True,
    seed= 1337
)

Found 9600 images belonging to 120 classes.
Found 2400 images belonging to 120 classes.


In [4]:
num_classes = len(train_generator.class_indices)
train_labels = train_generator.classes
train_labels = to_categorical(train_labels, num_classes=num_classes)
valid_labels = valid_generator.classes
valid_labels = to_categorical(valid_labels, num_classes=num_classes)
nb_train_samples = len(train_generator.filenames)
nb_valid_samples = len(valid_generator.filenames)

In [10]:
test_datagen = ImageDataGenerator(rescale=1/255.0)
test_generator = test_datagen.flow_from_directory(
                                    test_path,
                                    target_size=(img_width, img_height),
                                    color_mode= 'rgb',
                                    batch_size=batch_size,
                                    class_mode= 'categorical',
                                    shuffle=True,)
test_generator.reset()

Found 8580 images belonging to 120 classes.


# Model

**The following code are building model, you should optimized them.**



In [7]:
InceptionV3 = InceptionV3(include_top= False, input_shape= (img_width, img_height, channels), weights= 'imagenet')
# InceptionV3.summary()

model = Sequential()

for layer in InceptionV3.layers:
    layer.trainable = False

model.add(InceptionV3)
model.add(layers.Dropout(0.5))

model.add(layers.Flatten())
model.add(layers.BatchNormalization())
model.add(layers.Dense(1000,kernel_initializer='he_uniform'))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(120,activation='softmax'))
model.summary()

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
history=model.fit(train_generator, steps_per_epoch=50, epochs=10, verbose= 1, validation_data= valid_generator)

87924736/87910968 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 5, 5, 2048)        21802784  
                                                                 
 dropout (Dropout)           (None, 5, 5, 2048)        0         
                                                                 
 flatten (Flatten)           (None, 51200)             0         
                                                                 
 batch_normalization_94 (Bat  (None, 51200)            204800    
 chNormalization)                                                
                                                                 
 dense (Dense)               (None, 1000)              51201000  
                                                                 
 batch_normalization_95 (Bat  (None, 1000)             40

In [11]:
pred = model.predict(test_generator, verbose=2)
pred = np.argmax(pred,axis=1)

135/135 - 1625s - 1625s/epoch - 12s/step


In [14]:
test_accuracy = np.sum(np.array(pred))/len(pred)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 55.6003%


In [15]:
score = model.evaluate(test_generator, verbose=2)

135/135 - 952s - loss: 0.7448 - accuracy: 0.7872 - 952s/epoch - 7s/step
